In [5]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
from Trainer import train,train_acer, test
import gym as gym
from time import sleep
from time import time
from utils import preprocess

network_t = "cnn"
GAME_NAME="SpaceInvaders-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/invaders/25m4/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 1.5e-4
print(env.action_space)

Discrete(6)


/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [6]:
session = tf.Session()
tf.reset_default_graph()

with tf.Session() as sess:
    with tf.device('/cpu:0'):
        G_Agent=ACERNetwork(sess,env,"global",network_type=network_t)
        agent = ACERNetwork(sess,env,str(0),G_Agent, network_type=network_t)
        sess.run(tf.global_variables_initializer())
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))
        agent.update_target()
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))


/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[0.17137323 0.164999   0.16637686 0.1616064  0.16857705 0.16706751]]
[[0.16706552 0.16896443 0.16488437 0.17158034 0.16209365 0.16541173]]
[[0.16706552 0.16896443 0.16488437 0.17158034 0.16209365 0.16541173]]
[[0.16706552 0.16896443 0.16488437 0.17158034 0.16209365 0.16541173]]


In [3]:
def start_method(game_name=GAME_NAME, num_agents = 16, restore = False, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        #a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE)
            r_memory = ReplayBuffer(10000)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=5)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
            # Create a process for each worker
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=train_acer, args=(agents[thread_id],
                envs[thread_id], sess, thread_id, r_memory, 20, 0.99, 5000000, 200, saver, save_path)))
            # Start all the processes
            for p in processes:
                p.daemon = True
                p.start()

            # Until training is finished
            while not training_finished:
                sleep(0.05)

            # Join the processes, so we get this thread back.
            for p in processes:
                p.join()


In [ ]:
tf.reset_default_graph()
start_method()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Starting AgentStarting Agent 0
Starting Agent Starting Agent 3
2Starting Agent 4

Starting Agent 5
 1Starting Agent 6
Starting Agent 7
Starting Agent 8
Starting Agent 9

Starting Agent 10
Starting Agent 11
Starting AgentStarting Agent 13
Starting Agent 14
 Starting Agent 15
12
Worker  14 At  200  Running/Max:  0.0 0.0  Replay Ratio:  1.0
EPlen: 380.0 pi: [[0.2501606  0.25115427 0.24889779 0.24978732]]
Worker  4 At  400  Running/Max:  0.1 2.0  Replay Ratio:  1.0
EPlen: 370.19999999999993 pi: [[0.2506626  0.25119478 0.24918254 0.24896012]]
Worker  11 At  800  Running/Max:  0.2258625 2.0  Replay Ratio:  1.0
EPlen: 350.75 pi: [[0.24956127 0.2531584  0.25070223 0.24657807]]
Worker  0 At  1200  Running/Max:  0.256435890625 3.0  Replay Ratio:  1.0
EPlen: 330.32906 pi: [[0.25043663 0.2533927  0.24995887 0.24621181]]
Worker  3 At  1400  Running/Max:  0.504569375 4.0  Replay Ratio:  1.0
EPlen: 363.31300000000005 pi: [[0.25026044 0.25275785 0.24792425 0.24905755]]
Worker  13 At  1800  Running/Max

Worker  1 At  13200  Running/Max:  1.5926665541229075 Worker  5 At  13200  Running/Max:  1.4330440936011488 4.0  Replay Ratio:  1.0
5.0  Replay Ratio:  1.0
EPlen: 293.7196903517364 pi: [[0.23720409 0.25063884 0.29252914 0.21962792]]
EPlen: 288.62460093637105 pi: [[0.22418824 0.2443962  0.3272436  0.204172  ]]
Worker Worker  3 At  13400  Running/Max:  1.6561286853261008 4.0  Replay Ratio:  1.0
Worker   Worker 9 1 At  13400  Running/Max:  1.6130332264167622 5.0  Replay Ratio:  1.0
 6 At At  13400  Running/Max:  1.785456579379238 5.0  Replay Ratio:  1.0
 13400  Running/Max:  1.4471191293638057 7.0  Replay Ratio: EPlen: 310.2509762002814 pi: [[0.22217306 0.24155273 0.31587383 0.22040032]]
 1.0
EPlen:EPlen: 297.9636995109092 pi: [[0.23455308 0.24403685 0.29468274 0.2267273 ]] 
289.76214084273397 pi: [[0.20967625 0.2329203  0.34699205 0.21041135]]
EPlen: 285.2870635301008 pi: [[0.20373808 0.23600182 0.34803867 0.21222144]]
Worker  9 At  13600  Running/Max:  1.6613745628897623 5.0  Replay Rat

Worker  2 At  22000  Running/Max:  1.3551175967286029 5.0  Replay Ratio:  1.0
EPlen: 257.2566193838493 pi: [[0.18848945 0.26434916 0.30990463 0.23725666]]
Worker  2 At  22200  Running/Max:  1.3551175967286029 5.0  Replay Ratio:  1.0
EPlen: 257.2566193838493 pi: [[0.00611862 0.06489104 0.8599186  0.06907173]]
Worker  6 At  22400  Running/Max:  1.6326769615562915 7.0  Replay Ratio:  1.0
EPlen: 277.0770445582581 pi: [[0.01314331 0.08702202 0.798817   0.10101766]]
Worker  10 At  22800  Running/Max:  2.024627392634786 5.0  Replay Ratio:  1.0
EPlen: 318.18908849788323 pi: [[0.22316577 0.28822517 0.254658   0.23395102]]
Worker  11 At  23000  Running/Max:  2.4539425627280878 6.0  Replay Ratio:  1.0
EPlen: 352.22196571660083 pi: [[0.0291796  0.10094739 0.80633897 0.06353401]]
Worker  4 At  23200  Running/Max:  2.586502166959914 6.0  Replay Ratio:  1.0
EPlen: 346.1974375528365 pi: [[0.17405044 0.26604992 0.3366451  0.22325459]]
Worker  8 At  23400  Running/Max:  2.2292650639051272 4.0  Replay Ra

Worker  9 At  33200  Running/Max:  2.4665403100704775 5.0  Replay Ratio:  1.0
EPlen: 328.1442521089531 pi: [[0.05503513 0.14026901 0.24228671 0.56240916]]
Worker  1 At  33400  Running/Max:  2.6261527956770316 7.0  Replay Ratio:  1.0
EPlen: 345.17446117848067 pi: [[0.12363795 0.31203008 0.21212596 0.35220596]]
Worker  10 At  33600  Running/Max:  3.2821249348573946 8.0  Replay Ratio:  1.0
EPlen: 392.80511628921863 pi: [[0.16752231 0.3212528  0.21131624 0.2999087 ]]
Worker  5 At  33800  Running/Max:  2.47503048097043 10.0  Replay Ratio:  1.0
EPlen: 324.87242131048396 pi: [[0.18228836 0.2852766  0.14639431 0.38604075]]
Worker  4 At  34200  Running/Max:  3.1558613268724436 7.0  Replay Ratio:  1.0
EPlen: 393.0750740048866 pi: [[0.13893354 0.29040956 0.11039617 0.46026072]]
Worker Worker  12 At  34600  Running/Max:  3.1655115521623163 7.0  Replay Ratio:  7 At  34600  Running/Max:  2.85736732338235 9.0  Replay Ratio:  1.0
 1.0
EPlen: 361.6089787126738 pi: [[0.03324313 0.06437092 0.8616475  0.0

...

Worker  4 At  328600  Running/Max:  17.99654751594051 33.0  Replay Ratio:  1.0
Worker  14 At  328600  Running/Max:  15.819108566173627 34.0  Replay Ratio:  1.0
EPlen: 843.3445217530779 pi: [[3.5419546e-06 2.6517406e-03 9.1341692e-01 8.3927915e-02]]
EPlen: 848.0743781592953 pi: [[1.8407361e-07 9.9985003e-01 9.8097917e-06 1.3997608e-04]]
Worker  10 At  328800  Running/Max:  16.976877193659277 33.0  Replay Ratio:  1.0
Worker Worker  4 At  328800  Running/Max:  15 At  17.99654751594051 33.0  Replay Ratio:  1.0
 328800  Running/Max:  18.398906048114874 35.0 Worker Worker  Replay Ratio:  1.0
  114  At  328800  Running/Max:  15.978153137864945 34.0 At  Replay Ratio:  1.0
EPlen: EPlen: 890.7849640184168 pi: [[6.3248969e-11 1.1465976e-05 9.9998641e-01 2.0892367e-06]]
 328800EPlen:  Running/Max:  17.64738735555814 36.0 843.3445217530779  Replay Ratio:  1.0
 893.9625209455272 pi: [[1.1177214e-05 9.9198419e-01 3.8555948e-04 7.6190880e-03]]
pi: [[3.1016960e-05 1.6138008e-03 9.6811664e-01 3.0238472e

Worker  6 At  337600  Running/Max:  17.757689423667905 38.0  Replay Ratio:  1.0
EPlen: 876.7425653057646 pi: [[6.9829553e-10 7.6437274e-08 9.9999964e-01 2.6417993e-07]]
Worker  1 At  337800  Running/Max:  17.115744368309645 36.0  Replay Ratio:  1.0
EPlen: 848.6940078811915 pi: [[3.6588142e-06 7.2703564e-01 3.5496047e-04 2.7260566e-01]]
Worker  12 At  338200  Running/Max:  17.13514349381656 33.0  Replay Ratio:  1.0
EPlen: 876.8618772848245 pi: [[3.7633286e-06 9.7625762e-01 2.3297589e-02 4.4104565e-04]]
Worker  2 At  338800  Running/Max:  17.33217434628766 35.0  Replay Ratio:  1.0
EPlen: 847.0483980664822 pi: [[7.6514174e-04 7.7669501e-01 1.7496635e-01 4.7573436e-02]]
Worker  12 At  339000  Running/Max:  16.734467003169446 33.0  Replay Ratio:  1.0
EPlen: 850.4581206007078 pi: [[5.6370042e-10 5.2686846e-07 9.9999917e-01 3.3822769e-07]]
Worker  3 At  339200  Running/Max:  16.71682410880682 37.0  Replay Ratio:  1.0
EPlen: 887.5165148067965 pi: [[3.554155e-07 6.416660e-05 3.117286e-06 9.9993

Worker  14 At  347600  Running/Max:  16.274042780437394 34.0  Replay Ratio:  1.0
EPlen: 826.5520262512258 pi: [[5.2514909e-10 1.0171887e-05 9.9998975e-01 1.2207778e-07]]
Worker  2 At  348000  Running/Max:  17.553872176161 35.0  Replay Ratio:  1.0
EPlen: 872.1505371236752 pi: [[6.0981988e-06 9.9247038e-01 1.9236768e-04 7.3311077e-03]]
Worker  11 At  348200  Running/Max:  15.902470161057492 35.0  Replay Ratio:  1.0
Worker  15 At  348200  Running/Max:  17.432143230612333 35.0  Replay Ratio:  1.0
EPlen: 831.1714386348966 pi: [[1.6516903e-09 2.5256816e-06 9.9999750e-01 5.8669556e-08]]
EPlen: 832.5011150642207 pi: [[2.633790e-06 1.210507e-02 4.840414e-05 9.878439e-01]]
Worker  11 At  348400  Running/Max:  15.902470161057492 35.0  Replay Ratio:  1.0
EPlen: 831.1714386348966 pi: [[4.5214047e-06 1.6771188e-03 7.4925141e-05 9.9824345e-01]]
Worker Worker  8 At  348600  Running/Max:  18.467792352470276 44.0  Replay Ratio:  1.0
 14 At  348600  Running/Max:  16.360340641415522 34.0  Replay Ratio:  1

Worker  11 At  357000  Running/Max:  16.771821850746544 36.0  Replay Ratio:  1.0
EPlen: 905.2780453740199 pi: [[2.2355409e-06 1.4690455e-04 7.2029034e-06 9.9984360e-01]]
Worker Worker   13 At  357200  Running/Max:  16.996214471894902 47.0  Replay Ratio:  1.0
7 At  357200  Running/Max:  17.695587961238335 38.0  Replay Ratio:  1.0
EPlen: 867.2206867270303 pi: [[2.0951658e-09 9.9973673e-01 3.9824367e-06 2.5931245e-04]]
EPlen: 864.4161391380725 pi: [[6.2300161e-09 2.8545924e-07 4.5313332e-08 9.9999964e-01]]
Worker  2 At  357400  Running/Max:  16.491721600069738 35.0  Replay Ratio:  1.0
EPlen: 849.3314126407721 pi: [[2.9091566e-06 6.3921243e-02 1.3071233e-02 9.2300457e-01]]
Worker  0 At  357600  Running/Max:  17.113498574616766 41.0  Replay Ratio:  1.0
EPlen: 871.3121095242225 pi: [[1.3958957e-09 9.9840575e-01 1.1560405e-05 1.5827736e-03]]
Worker  12 At  357800  Running/Max:  Worker Worker Worker  0 16.532943268794902 33.0  Replay Ratio:  1.0
At  357800  Running/Max:  Worker  Worker  Worker

 pi: [[5.1739313e-10 9.9999356e-01 2.4841458e-07 6.1396040e-06]]
Worker  6 At  361800  Running/Max:  15.652652883034722 38.0  Replay Ratio:  1.0
EPlen: 808.5215993125873 pi: [[8.2832730e-06 1.8939349e-03 9.9728060e-01 8.1717782e-04]]
Worker  6 At  362200  Running/Max:  15.339019226938836 38.0  Replay Ratio:  1.0
EPlen: 776.7024954431959 pi: [[1.8376219e-06 9.9811220e-01 3.6614147e-05 1.8493608e-03]]
Worker  1 At  362400  Running/Max:  16.9382370570907 37.0  Replay Ratio:  1.0
EPlen: 859.2894084822013 pi: [[1.3932558e-06 9.9873668e-01 7.3082927e-05 1.1887802e-03]]
Worker  9 At  362600  Running/Max:  16.256600330017523 40.0  Replay Ratio:  1.0
EPlen: 846.1810706809483 pi: [[1.7797604e-05 9.9428517e-01 1.1977713e-03 4.4993320e-03]]
Worker  6 At  363000  Running/Max:  15.622068265591894 38.0  Replay Ratio:  1.0
EPlen: 799.0322458988762 pi: [[0.00229591 0.93833375 0.00970661 0.04966382]]
Worker Worker  6 At  363200  Running/Max:  15.622068265591894 38.0  Replay Ratio:  1.0
 9 Worker At  13 

In [7]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
                
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 2, True)))

            for p in processes:
                p.daemon = True
                p.start()

                
            for p in processes:
                p.join()


In [8]:
tf.reset_default_graph()
test_method()

/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from saves/invaders/25m4/checkpoints/model999654.cptk


/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run  0 Reward:  33.0
Run  1 Reward:  34.0


In [3]:
def start_method2(game_name=GAME_NAME, num_agents = 16, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    workers=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            r_memory = ReplayBuffer(10000)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=5)
            

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
            # Create a process for each worker
            for thread_id in range(num_agents):
                if thread_id == 0:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,"eval", r_memory, 20, 0.99, 1000000, 1))
                    processes.append(threading.Thread(target=workers[thread_id].work_and_eval_acer, args=(saver,TB_DIR)))
                else:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,str(thread_id), r_memory, 20, 0.99, 1000000, 1))
                    processes.append(threading.Thread(target=workers[thread_id].work_acer))
            # Start all the processes
            for p in processes:
                p.daemon = True
                p.start()
                
            # Join the processes, so we get this thread back.
            coord.join(processes)


In [ ]:
tf.reset_default_graph()
start_method2()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from saves/seaquest/15m4/checkpoints/model83616.cptk


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  worker_eval At  85982  Running/Max:  4.800000000000001 39.0  Replay Ratio:  1.0
pi: [[0.04920946 0.02396219 0.05921555 0.03115657 0.04524076 0.07426626
  0.04226107 0.02610819 0.05459629 0.03021411 0.03486921 0.04369945
  0.1024414  0.09603043 0.01756244 0.04723996 0.05974822 0.16217846]]
Saving Model
200.0
Worker  worker_eval At  88238  Running/Max:  8.220000000000002 39.0  Replay Ratio:  1.0
pi: [[0.05406791 0.02372779 0.05188944 0.04288781 0.05201452 0.09427733
  0.02955778 0.02645772 0.05664571 0.03757983 0.03719877 0.04556596
  0.07055597 0.10707165 0.01948447 0.03018419 0.08854423 0.13228896]]
Saving Model
400.0
Worker  worker_eval At  89797  Running/Max:  9.898000000000003 39.0  Replay Ratio:  1.0
pi: [[0.04983796 0.02414348 0.04834475 0.04346985 0.04440072 0.10207067
  0.03097499 0.02500352 0.06216594 0.04034347 0.03549611 0.04444873
  0.06723043 0.10301203 0.01828411 0.02783191 0.11443231 0.11850896]]
Saving Model
600.0
Worker  worker_eval At  91170  Running/Max:  11.0

5200.0
Worker  worker_eval At  132494  Running/Max:  27.732725965644327 45.0  Replay Ratio:  1.0
pi: [[0.02701499 0.02463092 0.03388128 0.03135099 0.03595113 0.13217436
  0.03065089 0.02178062 0.05071272 0.07039519 0.05086066 0.05246613
  0.03224126 0.10376033 0.05610805 0.01755228 0.16825448 0.06021363]]
Saving Model
5400.0
Worker  worker_eval At  134576  Running/Max:  28.559453369079897 45.0  Replay Ratio:  1.0
pi: [[0.02431859 0.02536988 0.03352113 0.03351374 0.03294284 0.12667058
  0.02921674 0.02417501 0.05652992 0.06235608 0.05173934 0.0490839
  0.03144582 0.1320688  0.04044602 0.01612183 0.16589852 0.06458127]]
Saving Model
5600.0
Worker  worker_eval At  136394  Running/Max:  28.703508032171907 45.0  Replay Ratio:  1.0
pi: [[0.01822356 0.01837558 0.02729164 0.0278313  0.02366593 0.13033342
  0.02451931 0.01791133 0.05268116 0.05299488 0.04850119 0.04735259
  0.02442417 0.14632131 0.03262764 0.01017325 0.22325741 0.07351432]]
Saving Model
5800.0
Worker  worker_eval At  138471  Ru

...

36400.0
Worker  worker_eval At  478247  Running/Max:  44.447005534306335 48.0  Replay Ratio:  1.0
pi: [[0.06284844 0.0512355  0.04510583 0.06683082 0.05825679 0.05823568
  0.05741451 0.07320285 0.05387114 0.05481598 0.04033491 0.07175313
  0.03363113 0.07502279 0.05293165 0.04159888 0.07069777 0.0322123 ]]
Saving Model
36600.0
Worker  worker_eval At  480813  Running/Max:  44.7023049808757 48.0  Replay Ratio:  1.0
pi: [[0.04655106 0.04979493 0.03333163 0.06902243 0.05065251 0.07966691
  0.04637447 0.07718799 0.06285862 0.0500761  0.02758249 0.07110959
  0.02253165 0.11939359 0.03496311 0.02550727 0.10068408 0.03271155]]
Saving Model
36800.0
Worker  worker_eval At  483411  Running/Max:  44.93207448278814 48.0  Replay Ratio:  1.0
pi: [[0.04516725 0.04349038 0.02985958 0.07307175 0.05304693 0.06319695
  0.05233919 0.10891062 0.06512333 0.06692324 0.03083279 0.05576224
  0.02100688 0.09445228 0.03207084 0.01779197 0.12003936 0.02691444]]
Saving Model
37000.0
Worker  worker_eval At  486004  

41600.0
Worker  worker_eval At  541772  Running/Max:  43.293489194884415 48.0  Replay Ratio:  1.0
pi: [[0.0301889  0.02639215 0.0275184  0.02869346 0.02509606 0.1109264
  0.04377996 0.03202526 0.12385531 0.03203541 0.02602484 0.17223413
  0.0162839  0.132457   0.03162458 0.0205124  0.1027836  0.01756825]]
Saving Model
41800.0
Worker  worker_eval At  544360  Running/Max:  43.56414027539598 48.0  Replay Ratio:  1.0
pi: [[0.0423761  0.04227661 0.05191648 0.05589092 0.05231135 0.04910707
  0.10173858 0.07883609 0.05868319 0.07938439 0.0512395  0.08825796
  0.03405331 0.05163713 0.04460146 0.03293805 0.05640243 0.02834928]]
Saving Model
42000.0
Worker  worker_eval At  546471  Running/Max:  42.80772624785638 48.0  Replay Ratio:  1.0
pi: [[0.04661495 0.04897262 0.05550337 0.06409046 0.06666517 0.03710359
  0.10902908 0.09281985 0.03763546 0.0935896  0.06363548 0.04914655
  0.04607737 0.03278742 0.03401112 0.04371078 0.03964401 0.03896315]]
Saving Model
42200.0
Worker  worker_eval At  549053  

46800.0
Worker  worker_eval At  603267  Running/Max:  42.00705150366039 48.0  Replay Ratio:  1.0
pi: [[0.02571699 0.0157135  0.03473988 0.02467033 0.01273622 0.00803717
  0.10240073 0.02506763 0.01593008 0.01859008 0.06151552 0.11732925
  0.0104589  0.0643798  0.37336951 0.02440008 0.05804177 0.00690255]]
Saving Model
47000.0
Worker  worker_eval At  605342  Running/Max:  41.40634635329435 48.0  Replay Ratio:  1.0
pi: [[0.00500292 0.02025664 0.00518076 0.01815159 0.01433381 0.0366829
  0.04495053 0.03976118 0.00366389 0.03904167 0.00524071 0.00638812
  0.01329985 0.09015117 0.00066436 0.57506084 0.02055549 0.0616135 ]]
Saving Model
47200.0
Worker  worker_eval At  607443  Running/Max:  40.96571171796492 48.0  Replay Ratio:  1.0
pi: [[0.03109328 0.05013209 0.01465235 0.05829324 0.03501771 0.09654285
  0.08948945 0.09677474 0.03895761 0.05949179 0.02503054 0.06020575
  0.014459   0.17692956 0.01810863 0.01541585 0.0807249  0.03868056]]
Saving Model
47400.0
Worker  worker_eval At  609691  R

52000.0
Worker  worker_eval At  667320  Running/Max:  44.17147258200122 48.0  Replay Ratio:  1.0
pi: [[0.08539533 0.03951905 0.06504718 0.07010651 0.064308   0.03744891
  0.0942753  0.16980553 0.0144233  0.03482384 0.05008753 0.11647359
  0.02989438 0.02926296 0.0266496  0.01486429 0.02838628 0.02922843]]
Saving Model
52200.0
Worker  worker_eval At  669652  Running/Max:  43.9543253238011 48.0  Replay Ratio:  1.0
pi: [[0.0060898  0.00415379 0.0024746  0.0060199  0.00497996 0.24080151
  0.00361893 0.00787934 0.07199044 0.00306489 0.00261564 0.14020382
  0.0014191  0.27250344 0.00607565 0.00078782 0.22239983 0.00292159]]
Saving Model
52400.0
Worker  worker_eval At  672209  Running/Max:  44.258892791421 48.0  Replay Ratio:  1.0
pi: [[0.06244539 0.05682097 0.02734899 0.07185179 0.0731229  0.09581576
  0.07842606 0.2055853  0.01035028 0.0384433  0.02576994 0.06735593
  0.03039022 0.05380753 0.01246587 0.01596108 0.03848521 0.03555344]]
Saving Model
52600.0
Worker  worker_eval At  674766  Run

57200.0
Worker  worker_eval At  730883  Running/Max:  44.846723278683086 49.0  Replay Ratio:  1.0
pi: [[0.06967814 0.04712228 0.08819737 0.07036176 0.0652269  0.02033208
  0.10156184 0.11108804 0.02478753 0.09681041 0.04484819 0.04736554
  0.0458682  0.02754579 0.03859676 0.01451314 0.04769525 0.03840076]]
Saving Model
57400.0
Worker  worker_eval At  733478  Running/Max:  45.06205095081478 49.0  Replay Ratio:  1.0
pi: [[0.07160751 0.04072132 0.09791244 0.06280224 0.04952717 0.02059726
  0.07379109 0.10810771 0.02249827 0.0725727  0.04489543 0.04747463
  0.03976025 0.03086806 0.08831508 0.01322227 0.08090792 0.03441868]]
Saving Model
57600.0
Worker  worker_eval At  736067  Running/Max:  45.255845855733305 49.0  Replay Ratio:  1.0
pi: [[0.07311115 0.04531448 0.11959481 0.05063502 0.04972797 0.03261296
  0.07592513 0.09718219 0.02270729 0.07169144 0.04469331 0.05654258
  0.05773179 0.03205256 0.04729291 0.02603832 0.04903827 0.04810786]]
Saving Model
57800.0
Worker  worker_eval At  738180